In [1]:
pip install langchain langchain-openai langchain-community chromadb pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 128.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 114.7 MB/s eta 

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader

# 1. API Key 설정 (디스코드 공지 참고하거나 개인 키 사용)
os.environ["OPENAI_API_KEY"] = ""

# 2. PDF 파일 로드 (샘플 RFP 파일 경로 지정)
# 실제 프로젝트에서는 반복문으로 여러 파일을 읽어야 합니다.
file_path = "/content/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf"
loader = PyPDFLoader(file_path)
raw_documents = loader.load()

print(f"총 {len(raw_documents)} 페이지를 로드했습니다.")

총 297 페이지를 로드했습니다.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 3. 텍스트 분할
# RFP는 문맥이 중요하므로 chunk_size를 너무 작게 잡지 않는 것이 좋습니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # 한 조각의 크기 (토큰 아님, 글자 수 기준)
    chunk_overlap=100,    # 조각 간 겹치는 구간 (문맥 유지용)
)

documents = text_splitter.split_documents(raw_documents)
print(f"문서를 {len(documents)}개의 청크로 분할했습니다.")

문서를 601개의 청크로 분할했습니다.


In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# 4. 임베딩 모델 및 벡터 DB 설정
# 비용 효율적인 text-embedding-3-small 사용 권장 [cite: 72]
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 5. 벡터 DB에 저장 (메모리 상에서 실행, 필요 시 persist_directory 설정하여 로컬 저장 가능)
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3} # 가장 유사한 문서 3개만 참조
)

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableSequence, RunnableParallel
from operator import itemgetter
from langchain_core.documents import Document

# 6. LLM 설정
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

# 7. 프롬프트 작성 (입찰메이트 컨설턴트 페르소나)
system_prompt = (
    "당신은 B2G 입찰 제안서 분석 전문가인 '입찰메이트'의 AI 컨설턴트입니다. "
    "아래 제공된 [Context]를 바탕으로 사용자의 질문에 답변하세요. "
    "문서에 없는 내용은 '문서에 해당 내용이 없습니다'라고 정직하게 말하세요."
    "\n\n"
    "[Context]:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# 💡 문서 리스트를 하나의 문자열로 포맷팅하는 함수 (create_stuff_documents_chain 대체 역할)
def format_docs(docs: list[Document]) -> str:
    """검색된 문서 리스트를 하나의 문자열로 결합합니다."""
    return "\n\n".join(doc.page_content for doc in docs)

# 8. LCEL 체인 구성 (Pipe '|' 연산자 사용)
# Input: {"input": "사용자 질문"}
# 1. 'input'은 'context' 키를 위해 retriever로 전달되고, 동시에 LLM 프롬프트의 'input'을 위해 유지됩니다.
# 2. retriever가 문서들을 검색합니다.
# 3. 검색된 문서들은 format_docs 함수를 통해 하나의 문자열로 포맷됩니다.
# 4. 최종적으로 LLM을 호출하여 답변을 생성합니다.

rag_chain = (
    # 1. 병렬 처리: context는 retriever와 format_docs를 통해, input은 입력 딕셔너리에서 'input' 키의 값만 가져옴
    RunnableParallel(
        context=itemgetter("input") | retriever | format_docs, # 쿼리를 retriever에 전달
        input=itemgetter("input") # 실제 쿼리 문자열을 다음 단계의 'input'으로 전달
    )
    | prompt
    | llm
)

# --- 테스트 실행 (retriever가 정의되어 있다는 가정 하에) ---
query = "현재 학사 행정 시스템의 문제는 무엇이고 어떻게 해결할 예정인가요?"
response = rag_chain.invoke({"input": query})
print(f"질문: {query}")
print(f"답변: {response.content}")

질문: 현재 학사 행정 시스템의 문제는 무엇이고 어떻게 해결할 예정인가요?
답변: 요약하면 현재 학사행정시스템의 주요 문제와 문서에 명시된 해결 방안은 다음과 같습니다.

문제점
- 학적정보의 파편화로 행정업무 비효율 발생
  - 편제(조직) 변경 이력 관리 기능 부재
  - 학적 정보 공유체계 미비 → 이중 작업·수작업 발생
- 서울·세종 캠퍼스 간 학사제도 차이를 반영하지 못함
  - 일부 기능(예: 핵심역량 연계 강의계획서 등록 등)이 캠퍼스별 차이를 처리하지 못함
- 포털과 별도 시스템(차세대 학사행정, 전자결재 등) 연계·접근 방식 미확정
  - 현재 포털에 있는 일부 서비스는 추후 별도 시스템 구축 방식에 따라 접근성 유지 필요
- 사용자 편의성(개인별 구성, 즐겨찾기 등) 고려된 연계 방법 미확정

해결 방안(문서에 명시된 계획)
- 양 캠퍼스 통합, 표준화된 학사행정시스템 구축
  - 표준화하되 조회/입력 등은 필요 시 서울캠퍼스와 세종캠퍼스 구분 처리가 가능하도록 구현
- 학적 정보 통합·공유체계 구축 및 이력관리 기능 추가
  - 편제변경 이력 관리 기능 구현으로 편제 관리의 이중 작업 제거
  - 학적정보의 일원화로 수작업·이중작업 감소
- 학사행정의 핵심 기능 구현(요구사항 SFR-학사-001 예시)
  - 학사일정관리: 캠퍼스·단위별 일정 등록·캘린더 관리, 신청 필요 일정의 기간 등록
  - 정원관리: 학과별 정원(내/외), 입학정원·편제정원 관리, 연도·학기별·단위별 관리, 직전학기 정원 데이터 복사 기능, 입학시스템 연계
  - 편제관리 및 편제변경 이력 관리
- 포털 연계 및 사용자 맞춤 구성 지원
  - 차후 별도 시스템에서 호출하지 않는 포털 서비스(예: 인사/급여 화면 등)는 구축 방식 확정 후에도 포털에서 동일 접근 가능
  - 연계 화면은 즐겨찾기 등으로 개인별 구성 가능하도록 제공
  - 사용자 편의성을 고려한 서비스 연계 방법은 관계자 협의를 통해 결정
- 학생이 자주 쓰는 학사정보는 바로 접근 가능하게 제공
  - 